# Time Series Comparison: Correlation, Derivatives, DTW, and PCA

Compare multiple time series from different workloads using four complementary analysis methods:
1. **Correlation Heatmap** - Overall similarity
2. **Derivative Correlation** - Rate-of-change similarity
3. **DTW Distance Matrix** - Temporal pattern similarity
4. **PCA Scatter Plot** - Feature-based clustering

## 1. Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy import interpolate, signal
from scipy.stats import zscore, pearsonr
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Try to import dtaidistance for faster DTW computation
try:
    from dtaidistance import dtw
    HAS_DTAIDISTANCE = True
except ImportError:
    HAS_DTAIDISTANCE = False
    print("Note: dtaidistance not available, will use slower DTW implementation")

# Set visual style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)

print("✓ All libraries imported successfully")

## 2. Load and Explore CSV Files

In [ ]:
# Configuration
DATASET_DIR = Path("/Users/anajafizadeh/projects/sunyibm/vllm-study/timeseries-models/vllm_datasets")
OUTPUT_DIR = Path("/Users/anajafizadeh/projects/sunyibm/vllm-study/plots/results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Find available CSV files
csv_files = sorted(list(DATASET_DIR.glob("*.csv")))
print(f"Found {len(csv_files)} CSV files in {DATASET_DIR}")
print("\nAvailable files:")
for i, f in enumerate(csv_files, 1):
    print(f"  {i:2d}. {f.name}")

In [ ]:
# Select files to compare (modify this list as needed)
# Option 1: Compare first N files
N_FILES = 6
selected_files = csv_files[:N_FILES]

# Option 2: Compare specific files
# selected_files = [
#     DATASET_DIR / "CPU_Usage-data-2025-10-02_00_22_34.csv",
#     DATASET_DIR / "Memory_Usage-data-2025-10-02_00_22_44.csv",
# ]

print(f"\nSelected {len(selected_files)} files for comparison:")
for f in selected_files:
    print(f"  - {f.stem}")

In [ ]:
# Load and explore data
dataframes = {}
for csv_file in selected_files:
    try:
        df = pd.read_csv(csv_file, usecols=["timestamp", "value"])
        if not df.empty:
            df["timestamp"] = pd.to_datetime(df["timestamp"])
            dataframes[csv_file.stem] = df
    except Exception as e:
        print(f"Error loading {csv_file.name}: {e}")

print(f"\nSuccessfully loaded {len(dataframes)} datasets")
print("\nDataset statistics:")
for name, df in dataframes.items():
    print(f"\n{name}:")
    print(f"  Shape: {df.shape}")
    print(f"  Value range: [{df['value'].min():.2f}, {df['value'].max():.2f}]")
    print(f"  Mean: {df['value'].mean():.2f}, Std: {df['value'].std():.2f}")
    print(f"  Sample:\n{df.head(3)}")

## 3. Normalize and Align Time Series Data

In [ ]:
# Create common time grid and align all series
min_time = min(df["timestamp"].min() for df in dataframes.values())
max_time = max(df["timestamp"].max() for df in dataframes.values())

# Convert to seconds since start
aligned_series = {}
time_offsets = {}

for name, df in dataframes.items():
    t_norm = (df["timestamp"] - df["timestamp"].iloc[0]).dt.total_seconds().values
    values = df["value"].values
    time_offsets[name] = (df["timestamp"].iloc[0] - min_time).total_seconds()
    
# Create common time grid
total_time = (max_time - min_time).total_seconds()
n_points = max(1000, max(len(df) for df in dataframes.values()))
time_grid = np.linspace(0, total_time, n_points)

print(f"Common time grid: {n_points} points from 0 to {total_time:.1f} seconds")
print(f"\nInterpolating all series to common grid...")

for name, df in dataframes.items():
    t_norm = (df["timestamp"] - df["timestamp"].iloc[0]).dt.total_seconds().values
    values = df["value"].values
    offset = time_offsets[name]
    
    # Shift time by offset
    t_shifted = t_norm + offset
    
    # Interpolate
    f = interpolate.interp1d(t_shifted, values, kind='cubic', 
                            bounds_error=False, fill_value='extrapolate')
    interpolated = f(time_grid)
    
    # Clip to reasonable values (handle extrapolation artifacts)
    interpolated = np.clip(interpolated, np.percentile(values, 1), np.percentile(values, 99))
    
    # Normalize (z-score)
    aligned_series[name] = (interpolated - np.mean(interpolated)) / (np.std(interpolated) + 1e-10)

print(f"✓ Aligned {len(aligned_series)} series to common grid")

In [ ]:
# Visualize aligned time series
fig, ax = plt.subplots(figsize=(14, 6))
for name, series in aligned_series.items():
    ax.plot(time_grid, series, label=name, linewidth=1.5, alpha=0.8)
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Normalized Value', fontsize=12)
ax.set_title('Aligned Time Series (Normalized)', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "00_aligned_time_series.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {OUTPUT_DIR / '00_aligned_time_series.png'}")

## 4. Generate Correlation Heatmap

In [ ]:
# Compute Pearson correlation matrix
labels = list(aligned_series.keys())
n = len(labels)
corr_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(n):
        s1 = aligned_series[labels[i]]
        s2 = aligned_series[labels[j]]
        corr_matrix[i, j] = np.corrcoef(s1, s2)[0, 1]

print("Correlation Matrix:")
print(pd.DataFrame(corr_matrix, index=labels, columns=labels).round(3))

In [ ]:
# Visualize correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="RdBu_r",
           xticklabels=labels, yticklabels=labels,
           vmin=-1, vmax=1, cbar_kws={"label": "Correlation"},
           ax=ax, square=True, linewidths=0.5)
ax.set_title('Correlation Heatmap (Normalized)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "01_correlation_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {OUTPUT_DIR / '01_correlation_heatmap.png'}")

## 5. Calculate and Visualize Derivative Correlation Heatmap

In [ ]:
# Compute derivatives (rate of change)
derivatives = {}
for name, series in aligned_series.items():
    deriv = np.gradient(series)
    derivatives[name] = deriv

# Compute correlation matrix of derivatives
deriv_corr_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        d1 = derivatives[labels[i]]
        d2 = derivatives[labels[j]]
        # Normalize derivatives
        d1_norm = (d1 - np.mean(d1)) / (np.std(d1) + 1e-10)
        d2_norm = (d2 - np.mean(d2)) / (np.std(d2) + 1e-10)
        deriv_corr_matrix[i, j] = np.corrcoef(d1_norm, d2_norm)[0, 1]

print("Derivative Correlation Matrix:")
print(pd.DataFrame(deriv_corr_matrix, index=labels, columns=labels).round(3))

In [ ]:
# Visualize derivative correlation heatmap
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(deriv_corr_matrix, annot=True, fmt=".2f", cmap="RdBu_r",
           xticklabels=labels, yticklabels=labels,
           vmin=-1, vmax=1, cbar_kws={"label": "Correlation"},
           ax=ax, square=True, linewidths=0.5)
ax.set_title('Derivative Correlation Heatmap (Rate of Change)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "02_derivative_correlation_heatmap.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {OUTPUT_DIR / '02_derivative_correlation_heatmap.png'}")

## 6. Compute DTW Distance Matrix

In [ ]:
def dtw_distance_simple(series1, series2):
    """Compute Dynamic Time Warping distance using dynamic programming."""
    n, m = len(series1), len(series2)
    dtw_matrix = np.full((n + 1, m + 1), np.inf)
    dtw_matrix[0, 0] = 0
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            cost = np.abs(series1[i-1] - series2[j-1])
            dtw_matrix[i, j] = cost + min(
                dtw_matrix[i-1, j],
                dtw_matrix[i, j-1],
                dtw_matrix[i-1, j-1]
            )
    
    return dtw_matrix[n, m]

print("Computing DTW distances... (this may take a moment)")

dtw_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(i, n):
        if HAS_DTAIDISTANCE:
            dist = dtw.distance_fast(aligned_series[labels[i]], aligned_series[labels[j]])
        else:
            dist = dtw_distance_simple(aligned_series[labels[i]], aligned_series[labels[j]])
        dtw_matrix[i, j] = dist
        dtw_matrix[j, i] = dist
        if (i+1) % 2 == 0:
            print(f"  Progress: {i+1}/{n}", end='\r')

print("\n✓ DTW computation complete")
print("\nDTW Distance Matrix:")
print(pd.DataFrame(dtw_matrix, index=labels, columns=labels).round(3))

In [ ]:
# Normalize DTW distances for better visualization
dtw_matrix_norm = dtw_matrix / np.max(dtw_matrix)

# Visualize DTW distance matrix
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(dtw_matrix_norm, annot=True, fmt=".2f", cmap="YlOrRd",
           xticklabels=labels, yticklabels=labels,
           cbar_kws={"label": "Normalized DTW Distance"},
           ax=ax, square=True, linewidths=0.5)
ax.set_title('DTW Distance Matrix (Temporal Patterns)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "03_dtw_distance_matrix.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {OUTPUT_DIR / '03_dtw_distance_matrix.png'}")

## 7. Perform PCA and Create Scatter Plot

In [ ]:
# Extract 12 statistical features from each time series
def extract_features(series):
    mean_val = np.mean(series)
    std_val = np.std(series)
    
    features = np.array([
        mean_val,
        std_val,
        np.min(series),
        np.max(series),
        np.median(series),
        np.percentile(series, 25),
        np.percentile(series, 75),
        np.mean(np.abs(np.gradient(series))),  # mean absolute derivative
        np.std(np.gradient(series)),  # std of derivative
        (np.max(series) - np.min(series)) / (mean_val + 1e-10),  # coefficient of variation
    ])
    return features

# Extract features for all series
features_list = []
for name in labels:
    features_list.append(extract_features(aligned_series[name]))

features_array = np.array(features_list)
feature_names = ['Mean', 'Std', 'Min', 'Max', 'Median', 'Q1', 'Q3', 'Mean|Deriv|', 'Std(Deriv)', 'CoV']

# Show feature values
print("Extracted Features:")
df_features = pd.DataFrame(features_array, index=labels, columns=feature_names)
print(df_features.round(3))

In [ ]:
# Standardize features and apply PCA
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_array)

pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

print(f"PCA Explained Variance Ratio:")
for i, var in enumerate(pca.explained_variance_ratio_):
    print(f"  PC{i+1}: {var:.1%}")
print(f"  Total: {sum(pca.explained_variance_ratio_):.1%}")

In [ ]:
# Create PCA scatter plot
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(features_pca[:, 0], features_pca[:, 1],
                     s=300, alpha=0.7, c=range(len(labels)),
                     cmap='tab10', edgecolors='black', linewidth=2)

# Add labels
for i, label in enumerate(labels):
    ax.annotate(label, (features_pca[i, 0], features_pca[i, 1]),
               xytext=(10, 10), textcoords='offset points',
               fontsize=9, bbox=dict(boxstyle='round,pad=0.5',
               facecolor='yellow', alpha=0.4))

ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)', fontsize=12)
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)', fontsize=12)
ax.set_title('PCA Scatter Plot (Feature-Based Clustering)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.3, linewidth=1)
ax.axvline(x=0, color='k', linestyle='--', alpha=0.3, linewidth=1)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "04_pca_scatter_plot.png", dpi=300, bbox_inches='tight')
plt.show()
print(f"✓ Saved to {OUTPUT_DIR / '04_pca_scatter_plot.png'}")

## 8. Comparative Analysis and Insights

In [ ]:
print("="*80)
print("COMPARATIVE ANALYSIS SUMMARY")
print("="*80)

# Analysis 1: Correlation
print("\n1. CORRELATION ANALYSIS")
print("-" * 80)
max_corr_idx = np.unravel_index(np.argmax(np.triu(corr_matrix, k=1)), corr_matrix.shape)
if max_corr_idx[0] != max_corr_idx[1]:
    print(f"Most correlated pair: {labels[max_corr_idx[0]]} <-> {labels[max_corr_idx[1]]}")
    print(f"  Correlation coefficient: {corr_matrix[max_corr_idx[0], max_corr_idx[1]]:.3f}")

min_corr_idx = np.unravel_index(np.argmin(np.triu(corr_matrix) + np.diag([2]*n)), corr_matrix.shape)
if min_corr_idx[0] != min_corr_idx[1]:
    print(f"\nLeast correlated pair: {labels[min_corr_idx[0]]} <-> {labels[min_corr_idx[1]]}")
    print(f"  Correlation coefficient: {corr_matrix[min_corr_idx[0], min_corr_idx[1]]:.3f}")

# Analysis 2: Derivative Correlation
print("\n2. DERIVATIVE CORRELATION ANALYSIS")
print("-" * 80)
max_deriv_idx = np.unravel_index(np.argmax(np.triu(deriv_corr_matrix, k=1)), deriv_corr_matrix.shape)
if max_deriv_idx[0] != max_deriv_idx[1]:
    print(f"Most similar rate-of-change: {labels[max_deriv_idx[0]]} <-> {labels[max_deriv_idx[1]]}")
    print(f"  Derivative correlation: {deriv_corr_matrix[max_deriv_idx[0], max_deriv_idx[1]]:.3f}")

# Analysis 3: DTW Distances
print("\n3. DTW DISTANCE ANALYSIS")
print("-" * 80)
min_dtw_idx = np.unravel_index(np.argmin(np.triu(dtw_matrix, k=1) + np.diag([np.inf]*n)), dtw_matrix.shape)
if min_dtw_idx[0] != min_dtw_idx[1]:
    print(f"Most similar temporal patterns: {labels[min_dtw_idx[0]]} <-> {labels[min_dtw_idx[1]]}")
    print(f"  DTW distance: {dtw_matrix[min_dtw_idx[0], min_dtw_idx[1]]:.3f}")

# Analysis 4: PCA Clustering
print("\n4. PCA FEATURE ANALYSIS")
print("-" * 80)
print("Principal Component Loadings:")
for i in range(2):
    print(f"\n  PC{i+1} ({pca.explained_variance_ratio_[i]:.1%} variance):")
    components = pca.components_[i]
    top_idx = np.argsort(np.abs(components))[-3:][::-1]
    for idx in top_idx:
        print(f"    {feature_names[idx]}: {components[idx]:.3f}")

print("\n" + "="*80)

In [ ]:
# Create a comprehensive summary table
print("\nDETAILED COMPARISON SUMMARY")
print("="*80)

summary_data = []
for i, label in enumerate(labels):
    summary_data.append({
        'Series': label,
        'Mean': f"{aligned_series[label].mean():.3f}",
        'Std': f"{aligned_series[label].std():.3f}",
        'Avg Corr': f"{np.mean(corr_matrix[i]):.3f}",
        'Avg DTW': f"{np.mean(dtw_matrix[i]):.3f}",
        'PC1': f"{features_pca[i, 0]:.3f}",
        'PC2': f"{features_pca[i, 1]:.3f}"
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

print(f"\n✓ All visualizations saved to: {OUTPUT_DIR}")